<a href="https://colab.research.google.com/github/lsteffenel/CHPS0704/blob/main/TP2/TP2.2-MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Démarrage

Pour ce notebook, nous allons définir quelques paramètres pour la visualisation, ainsi qu'une fonction pour enregistrer des images.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# MNIST

Ici nous allons télécharger les images MNIST à partir de sklearn. Il est aussi possible de les avoir ailleurs (par exemple, un répertoire d'images), mais vu que c'est un dataset classique, autant gagner du temps.

Les images seront stockées directement dans des matrices.

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

In [ ]:
X, y = mnist["data"], mnist["target"]
X.shape

In [ ]:
y.shape

In [ ]:
y = y.astype(np.uint8)

La partie "features" du dataset (X) contient 784 colonnes. Ceci est du au fait que l'image fait 28x28 pixels, mais qu'on les représente comme un seul array de 784 éléments

In [ ]:
28*28

Allons voir comment ressemblent les images. Dans ce paragraphe on imprime la première image (X[0]), vous pouvez changer l'index pour afficher d'autres chiffres.

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap=mpl.cm.binary)
plt.axis("off")

save_fig("some_digit_plot")
plt.show()

Bien sûr, l'image est accompagnée d'un label (dans le dataframe Y). Voyons à quel chiffre correspond cette image :

In [ ]:
y[0]

Afin de simplifier la visualisation, les prochaines paragraphes définissent des fonctions pour l'affichage d'une ou de plusieurs images.

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")

In [ ]:
# EXTRA
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    # This is equivalent to n_rows = ceil(len(instances) / images_per_row):
    n_rows = (len(instances) - 1) // images_per_row + 1

    # Append empty images to fill the end of the grid, if needed:
    n_empty = n_rows * images_per_row - len(instances)
    padded_instances = np.concatenate([instances, np.zeros((n_empty, size * size))], axis=0)

    # Reshape the array so it's organized as a grid containing 28×28 images:
    image_grid = padded_instances.reshape((n_rows, images_per_row, size, size))

    # Combine axes 0 and 2 (vertical image grid axis, and vertical image axis),
    # and axes 1 and 3 (horizontal axes). We first need to move the axes that we
    # want to combine next to each other, using transpose(), and only then we
    # can reshape:
    big_image = image_grid.transpose(0, 2, 1, 3).reshape(n_rows * size,
                                                         images_per_row * size)
    # Now that we have a big image, we just need to show it:
    plt.imshow(big_image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(9,9))
example_images = X[:100]
plot_digits(example_images, images_per_row=10)
save_fig("more_digits_plot")
plt.show()

# Découpage des données

Comme les chiffres sont déjà bien mélangés, nous allons juste découper les datasets en Train et Test

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

## Question : sauriez-vous faire la même chose en utilisant la fonction `train_test_split` de ScikitLearn ?

# Un Classifieur Binaire

Pour simplifier l'apprentissage, nous allons choisir juste un chiffre (par exemple **5**) et créer un classifieur où la question sera "Est-ce un 5 ou pas".

On commence donc par créer des dataframes avec seulement le chiffre 5).

In [ ]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

Nous allons utiliser un autre algorithme non vu en cours. Le pricipe est le même que pour des arbres de décision ou des SVM, vous pouvez plus tard comparer les algorithmes.

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(X_train, y_train_5)

In [ ]:
sgd_clf.predict([X[500]])

# Analyse des métriques avec une validation croisée

Dans cette partie, nous allons utiliser une nouvelle technique appellée validation croisée. Au lieu de tester sur le groupe "test" uniquement, nous allons faire 5 sous-groupes, chacun évalué séparemment.

In [ ]:
from sklearn.model_selection import cross_val_score


cross_val_score(sgd_clf,X_train,y_train_5,cv=5,scoring="accuracy")


Les valeurs obtenus sont intéresants (>94%), mais ils sont trompeurs. En effet, le chiffre 5 ne représente qu'environ 10% des données. On peut vérifier ce "biais" avec un modèle bidon qui retourne "pas 5" systèmatiquement.

In [ ]:
from sklearn.base import BaseEstimator

class Never5(BaseEstimator):
    def fit(self,X,y=None):
        return self
    def predict(self,X):
        return np.zeros((len(X),1),dtype=bool)

In [ ]:
never_5=Never5()
cross_val_score(never_5,X_train,y_train_5,cv=5,scoring="accuracy")

Comme dans notre modèle, on voit une accuracy importante (>90%) alors que c'est un modèle sans aucun intérêt. Voilà donc une raison de ne pas croire uniquement à Accuracy.

# Matrice de confusion

Afin de mieux observer le modèle, nous pouvons utiliser la matrice de confusion.

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_5, y_train_pred)

On peut la montrer graphiquement aussi :

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

_=ConfusionMatrixDisplay.from_predictions(y_train_5,y_train_pred)

Chaque ligne de la matrice de confusion contient la classe réelle tandis que les colonnes sont constituées de la classe prédite.

Ici, la première ligne représente la classe non-5 réelle (la fausse classe). Plus de 53000  d'entre eux ont été correctement classés comme non-5 (vrai négatif) tandis que moins de 1000 ont été classés comme 5 (faux positif).

De même, la deuxième ligne contient la classe 5 réelle (la vraie classe). Un peu plus de 1800 ont été classés comme non-5 (faux négatif) tandis que 3530 ont été classés comme 5 (vrai positif).

# Precision, Recall et F1-Score

Afin de balancer le résultat de Accuracy, nous pouvons regarder ceux d'autres métriques, telles que le Recall, la Précision ou le Score F1.

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)

In [ ]:
recall_score(y_train_5, y_train_pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)

# Exercice

Maintenant, créer un RandomForestClassifier et entraîner de la même manière.
Comparez les métriques Accuracy, Recall, Precision et F1-Score obtenues.